In [1]:
import numpy as np
import json

In [15]:
x = np.array([0, 0, -0.068, -0.137, -0.137, -0.137, -0.162])
y = np.array([0, 0, 0, 0, 0.2, 0.39, 0.463])
z = np.array([0.12, 0.32, 0.509, 0.696, 0.7, 0.7, 0.687])
roll = np.array([0, 0, 0.349, 1.569, -1.57, -1.694, -1.694])
pitch = np.array([0, -0.349, 0, -1.222, -0.348, -0.362, -0.326])
yaw = np.array([0, 0, -1.571, -3.14, 0, 0.372, 0.372])

axes = np.array([[0, 0, 1], [0, -1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, -1, 0], [0, 0, 1]])
origins = np.array([x, y, z]).T
angles_rpy = np.array([roll, pitch, yaw]).T

In [30]:
def rotate(angles_rpy, vector):
    gamma = angles_rpy[0]
    beta = angles_rpy[1]
    alpha = angles_rpy[2]
    
    sin = np.sin
    cos = np.cos
    
    a = np.array([[cos(alpha)*cos(beta), cos(alpha)*sin(beta)*sin(gamma) - sin(alpha)*cos(gamma), cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma)],
                 [sin(alpha)*cos(beta), sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma), sin(alpha)*sin(beta)*cos(gamma) - cos(alpha)*sin(gamma)],
                 [-sin(beta), cos(beta)*sin(gamma), cos(beta)*cos(gamma)]])
    
    b = a.dot(vector)
    
    return b / np.linalg.norm(b)


def build_jacobian(origins, angles_rpy, axes):
    j0 = np.concatenate([np.cross([0, 0, 1], origins[0]), [0, 0, 1]])
    j = np.array([np.concatenate([np.cross(rotate(angles_rpy[i], axes[i]), origin - origins[i]), rotate(angles_rpy[i], axes[i])]) for i, origin in enumerate(origins[1:])])
    
    return np.concatenate([[j0], j]).T

In [31]:
jacobian = build_jacobian(origins, angles_rpy, axes)

np.set_printoptions(precision=3, linewidth=120)
print(jacobian)

[[ 0.000e+00  0.000e+00 -1.890e-01  1.302e-05 -6.835e-02 -1.422e-04 -6.758e-02]
 [ 0.000e+00  0.000e+00 -0.000e+00 -8.942e-04 -3.759e-03  0.000e+00 -2.804e-02]
 [ 0.000e+00  0.000e+00 -6.800e-02  4.806e-06  1.880e-01 -5.160e-05 -2.749e-02]
 [ 0.000e+00  0.000e+00  0.000e+00 -3.420e-01  9.398e-01 -2.716e-04 -3.721e-01]
 [ 0.000e+00  0.000e+00 -1.000e+00  6.965e-05 -2.996e-04  1.000e+00 -1.326e-02]
 [ 1.000e+00  1.000e+00  0.000e+00  9.397e-01  3.418e-01  7.486e-04  9.281e-01]]


In [32]:
jacobian_inv = np.linalg.inv(jacobian[:, 1:])
print(jacobian_inv)

[[ 4.970e+00 -1.866e+00 -1.381e+01  2.753e+00 -6.846e-06  1.000e+00]
 [-4.816e+00  1.336e+01 -1.309e+00 -3.513e-02 -7.621e-04 -0.000e+00]
 [-5.054e+00  3.854e+01  1.406e+01 -3.025e+00 -8.149e-04 -0.000e+00]
 [-1.687e+00 -5.509e-01  4.689e+00  1.442e-03  2.374e-06  0.000e+00]
 [-4.811e+00  1.287e+01 -1.323e+00 -3.364e-02  9.992e-01 -0.000e+00]
 [ 3.873e-01 -3.682e+01 -1.077e+00  9.627e-02  2.567e-05 -0.000e+00]]


In [34]:
force = np.concatenate([rotate(angles_rpy[-1], [0, 0, 1]), [0, 0, 0]])
torque = jacobian_inv.dot(force)

print('Force vector: {}'.format(force))
print('Joint torques: {}'.format(torque))

Force vector: [-0.324  0.939 -0.116  0.     0.     0.   ]
Joint torques: [ -1.754  14.253  36.183  -0.516  13.792 -34.567]
